In [1]:
!pip install peft # parameter efficient fine tune
!pip install accelerate # for optimize hard ware utlization
!pip install bitsandBytes
!pip install transformers
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [2]:
!pip install GPUtil # show gpu utilization

  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7392 sha256=f3a7aabeac746dd18b9a1f979eb8086c3325371bf9c224cb726b276f73117768
  Stored in directory: /root/.cache/pip/wheels/2b/4d/8f/55fb4f7b9b591891e8d3f72977c4ec6c7763b39c19f0861595
Successfully built GPUtil


In [3]:
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [4]:
import torch,transformers
import GPUtil
import os

GPUtil.showUtilization()

if torch.cuda.is_available():
    print("GPU is available")
else:
    print("GPU is not available, using CPU instead")

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
GPU is available


In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, LlamaTokenizer
from huggingface_hub import notebook_login
from datasets import load_dataset
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

In [6]:
if "COLAB_GPU" in os.environ:
  from google.colab import output
  output.enable_custom_widget_manager()

In [ ]:
if "COLAB_GPU" in os.environ:
  !huggingface-cli login
else:
  notebook_login()

In [8]:
model_id='NousResearch/Llama-2-7b-chat-hf'

bnb_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,  # doing double quantization
    bnb_4bit_quant_type='nf4' # for normal distributions
)

In [ ]:
model=AutoModelForCausalLM.from_pretrained(model_id,quantization_config=bnb_config,device_map={"":0})

In [10]:
from datasets import load_dataset

dataset_name='PhilSad/standfordsentiment'
# new_model = "Llama-2-7b-finetune"

dataset = load_dataset(dataset_name)



standford_train.csv:   0%|          | 0.00/786k [00:00<?, ?B/s]

standford_test.csv:   0%|          | 0.00/194k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5104 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1276 [00:00<?, ? examples/s]

In [11]:
print(dataset['train'])

Dataset({
    features: ['text', 'label'],
    num_rows: 5104
})


In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [13]:
config = LoraConfig(
    lora_alpha=64,
    lora_dropout=0.06,
    r=8,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
)

In [14]:
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=1024,
        padding="max_length",
    )

In [15]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/5104 [00:00<?, ? examples/s]

Map:   0%|          | 0/1276 [00:00<?, ? examples/s]

In [16]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 5104
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1276
    })
})

In [17]:
model.gradient_checkpointing_enable()
model=prepare_model_for_kbit_training(model)

In [18]:
training_arguments = transformers.TrainingArguments(
     output_dir="./finetunedModel",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=2,
        num_train_epochs=20,
        learning_rate=1e-4,
        max_steps=30,
        bf16=False,
        optim="paged_adamw_8bit",
        logging_dir="./log",
        save_strategy="epoch",
        save_steps=50,
        logging_steps=10,
        remove_unused_columns=False,
        report_to=[]
)

In [19]:

from trl import SFTTrainer

trainer=SFTTrainer(
    model=model,
    train_dataset=tokenized_dataset['train'],
    peft_config=config,
    args=training_arguments,

)

Truncating train dataset:   0%|          | 0/5104 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [20]:
model.config.use_cache=False
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,1.230500
20,0.131100
30,0.109200


TrainOutput(global_step=30, training_loss=0.4902902404467265, metrics={'train_runtime': 672.0928, 'train_samples_per_second': 0.179, 'train_steps_per_second': 0.045, 'total_flos': 4886199407738880.0, 'train_loss': 0.4902902404467265})

## Testing


In [38]:
trainer.model.save_pretrained("./finetunedModel")
tokenizer.save_pretrained("./finetunedModel")



('./finetunedModel/tokenizer_config.json',
 './finetunedModel/special_tokens_map.json',
 './finetunedModel/tokenizer.model',
 './finetunedModel/added_tokens.json',
 './finetunedModel/tokenizer.json')

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("./finetunedModel", device_map={"": "cpu"}, offload_folder="offload")
tokenizer = AutoTokenizer.from_pretrained("./finetunedModel")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
from datasets import load_dataset

# If not already loaded
dataset = load_dataset("PhilSad/standfordsentiment")

# If your test data is already tokenized, use the raw text column
for i in range(5):
    text = dataset["test"][i]["text"]
    inputs = tokenizer(text, return_tensors="pt").to("cpu")

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=50,
            do_sample=False,
            temperature=0.7,
            top_p=0.95
        )

    print(f"\nInput: {text}")
    print("Output:", tokenizer.decode(output[0], skip_special_tokens=True))



The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Input: <cls> Elvira fans could hardly ask for more . <sep> The sentiment is neutral. <sep>
Output: <cls> Elvira fans could hardly ask for more . <sep> The sentiment is neutral. <sep>


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Input: <cls> The story is also as unoriginal as they come , already having been recycled more times than I 'd care to count . <sep> The sentiment is neutral. <sep>
Output: <cls> The story is also as unoriginal as they come , already having been recycled more times than I 'd care to count . <sep> The sentiment is neutral. <sep>


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Input: <cls> Earnest and tentative even when it aims to shock . <sep> The sentiment is negative. <sep>
Output: <cls> Earnest and tentative even when it aims to shock . <sep> The sentiment is negative. <sep>


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Input: <cls> The smash 'em - up , crash 'em - up , shoot 'em - up ending comes out of nowhere substituting mayhem for suspense . <sep> The sentiment is negative. <sep>
Output: <cls> The smash 'em - up , crash 'em - up , shoot 'em - up ending comes out of nowhere substituting mayhem for suspense . <sep> The sentiment is negative. <sep>

Input: <cls> The Santa Clause 2 is a barely adequate babysitter for older kids , but I 've got to give it thumbs down . <sep> The sentiment is negative. <sep>
Output: <cls> The Santa Clause 2 is a barely adequate babysitter for older kids , but I 've got to give it thumbs down . <sep> The sentiment is negative. <sep>
